# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
import scipy
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
from secret import ALPHA_VANTAGE_API_TOKEN
from secret import POLYGON_API_KEY
stocks = pd.read_csv('sp_500_stocks.csv')
print(type(stocks))


<class 'pandas.core.frame.DataFrame'>


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
# symbol = 'NVDA'
# date = '2024-05-28'
# historical_price = requests.get(f"https://api.polygon.io/v1/open-close/{symbol}/{date}?adjusted=true&apiKey={POLYGON_API_KEY}")
# current_price = requests.get(f"https://api.polygon.io/v2/aggs/ticker/{symbol}/prev?adjusted=true&apiKey={POLYGON_API_KEY}")


In [4]:
# current_price = current_price.json()
# historical_price = historical_price.json()

# percent_change = current_price.json()['results']['c']/historical_price.json()['open']
# percent_change


In [5]:
# percent_change = int(np.floor(((current_price['results'][0]['c']/historical_price['open']) - 1) * 100))
# current_price


## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
all_data = []
columns_index = ['Ticker', 'Current Price', 'Historical Price', 'Annual Percentage Change', 'Shares to Buy']
generator = np.random.default_rng()
for stock in stocks['Ticker']:
    # print(stock)
    historical_price = float(generator.integers(50, 500))
    current_price = historical_price + float(generator.integers(-0.2 * historical_price, 0.25 * historical_price))
    # print(f'current_price: {current_price}')
    # print(f'historical_price: {historical_price}')
    percent_change = int(np.floor((((current_price/historical_price) - 1) * 100)))
    stock_data = [stock, current_price, historical_price, percent_change, 'N/A']
    series = pd.Series(data = stock_data, index = columns_index)
    # print(f'Series: {series.values}')
    all_data.append(series)
# print(f'all_data: {all_data}')
# print(all_data)
final_df = pd.DataFrame(data = all_data, columns = columns_index)
final_df

,Ticker,Current Price,Historical Price,Annual Percentage Change,Shares to Buy
0,A,299.0,282.0,6,N/A
1,AAL,290.0,244.0,18,N/A
2,AAP,549.0,452.0,21,N/A
3,AAPL,336.0,298.0,12,N/A
4,ABBV,283.0,325.0,-13,N/A
...,...,...,...,...,...
500,YUM,158.0,143.0,10,N/A
501,ZBH,256.0,292.0,-13,N/A
502,ZBRA,424.0,350.0,21,N/A
503,ZION,478.0,399.0,19,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_df.sort_values(by=['Annual Percentage Change'], inplace = True, ascending = False, ignore_index = True)
final_df = final_df.loc[0:49]
final_df

,Ticker,Current Price,Historical Price,Annual Percentage Change,Shares to Buy
0,SNPS,443.0,356.0,24,N/A
1,FMC,384.0,309.0,24,N/A
2,PFG,264.0,212.0,24,N/A
3,DRE,352.0,283.0,24,N/A
4,BAC,315.0,254.0,24,N/A
5,ARE,139.0,112.0,24,N/A
6,ALB,586.0,474.0,23,N/A
7,NOV,394.0,320.0,23,N/A
8,GIS,585.0,472.0,23,N/A
9,FTV,254.0,205.0,23,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
def get_portfolio_size():
    portfolio_size = ''
    while not type(portfolio_size) is int:
        try:
            portfolio_size = int(input('What is your portfolio size?'))
        except ValueError:
            print('Please enter an integer')
    return portfolio_size

def calculate_shares(portfolio_size, stocks_df):
    position_size = portfolio_size/len(stocks_df)
    for row in range(len(stocks_df)):
        price = stocks_df.at[row, 'Current Price']
        shares_to_buy = int(np.floor(position_size/price))
        stocks_df.at[row, 'Shares to Buy'] = shares_to_buy


In [10]:
portfolio_size = get_portfolio_size()
calculate_shares(portfolio_size, final_df)
final_df

What is your portfolio size? 32664


,Ticker,Current Price,Historical Price,Annual Percentage Change,Shares to Buy
0,SNPS,443.0,356.0,24,1
1,FMC,384.0,309.0,24,1
2,PFG,264.0,212.0,24,2
3,DRE,352.0,283.0,24,1
4,BAC,315.0,254.0,24,2
5,ARE,139.0,112.0,24,4
6,ALB,586.0,474.0,23,1
7,NOV,394.0,320.0,23,1
8,GIS,585.0,472.0,23,1
9,FTV,254.0,205.0,23,2


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [31]:
from statistics import mean
hqm_data = []
column_index = [
    'Ticker', 'Current Price', 'HQM Score','Shares to Buy', 
    'Year Price', 'One Year Returns', 'One Year Percentile Returns',
    'Six-Month Price', 'Six-Month Returns', 'Six-Month Percentile Returns',
    'Three-Month Price', 'Three-Month Returns', 'Three-Month Percentile Returns',
    'One-Month Price', 'One-Month Returns', 'One-Month Percentile Returns'
    ]
for stock in stocks['Ticker']:
    year_price = generator.integers(25, 150)
    six_month_price = year_price + generator.integers(-0.2 * year_price, 0.25 * year_price)
    three_month_price = six_month_price + generator.integers(-0.1 * six_month_price, 0.15 * six_month_price)
    one_month_price = three_month_price + generator.integers(-0.05 * three_month_price, 0.1 * three_month_price)
    current_price = historical_price + generator.integers(-0.1 * historical_price, 0.15 * historical_price)

    prices = [year_price, six_month_price, three_month_price, one_month_price, current_price]

    one_year_returns = int(np.floor(((current_price/year_price) - 1) * 100))
    six_month_returns = int(np.floor(((current_price/six_month_price) - 1) * 100))
    three_month_returns = int(np.floor(((current_price/three_month_price) - 1) * 100))
    one_month_returns = int(np.floor(((current_price/one_month_price) - 1) * 100))

    returns = [one_year_returns, six_month_returns, three_month_returns, one_month_returns]

    percentile_returns = scipy.stats.percentileofscore(returns, returns)

    hqm_score = mean(percentile_returns)

    stock_data = pd.Series(
        data = [
            stock, current_price, hqm_score, 'N/A',
            prices[0], returns[0], percentile_returns[0],
            prices[1], returns[1], percentile_returns[1],
            prices[2], returns[2], percentile_returns[2],
            prices[3], returns[3], percentile_returns[3]
        ], 
        index = column_index
    )

    hqm_data.append(stock_data)
hqm_df = pd.DataFrame(data  = hqm_data, columns = column_index)

hqm_df     
                    
    
    

,Ticker,Current Price,HQM Score,Shares to Buy,Year Price,One Year Returns,One Year Percentile Returns,Six-Month Price,Six-Month Returns,Six-Month Percentile Returns,Three-Month Price,Three-Month Returns,Three-Month Percentile Returns,One-Month Price,One-Month Returns,One-Month Percentile Returns
0,A,172.0,62.5,N/A,146,17,75.0,161,6,25.0,150,14,50.0,144,19,100.0
1,AAL,174.0,62.5,N/A,41,324,100.0,46,278,62.5,48,262,25.0,46,278,62.5
2,AAP,193.0,62.5,N/A,53,264,100.0,59,227,75.0,60,221,50.0,61,216,25.0
3,AAPL,194.0,62.5,N/A,65,198,100.0,79,145,25.0,73,165,62.5,73,165,62.5
4,ABBV,168.0,62.5,N/A,27,522,25.0,25,572,50.0,23,630,100.0,24,600,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,163.0,62.5,N/A,126,29,75.0,125,30,100.0,134,21,25.0,130,25,50.0
501,ZBH,181.0,62.5,N/A,89,103,25.0,85,112,50.0,77,135,75.0,74,144,100.0
502,ZBRA,178.0,62.5,N/A,111,60,100.0,127,40,50.0,123,44,75.0,130,36,25.0
503,ZION,187.0,62.5,N/A,90,107,25.0,85,120,75.0,84,122,100.0,87,114,50.0


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: